# Домашнее задание №2 Чебыкина Артёма

### №1-№2

Загружаем библиотеки и считываем данные

In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [3]:
data = pd.read_pickle("data/train.pickle")

Putting data into a needed format, creating DataFrames and combining them

In [4]:
arr = np.array(data['features'])
new_features_data = arr.reshape(34799,32*32*3)
x_features = pd.DataFrame(new_features_data)
x_features.shape

(34799, 3072)

In [5]:
x_features = (x_features - x_features.mean()) / x_features.std()

In [6]:
scaler = MinMaxScaler()
scaler.fit(x_features)
x_features = scaler.transform(x_features)

Labels are out targets

In [7]:
y_df = data['labels']
y_df.shape

(34799,)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x_features, y_df, train_size=0.8)

In [9]:
x_train.shape, x_test.shape

((27839, 3072), (6960, 3072))

Обучим все необходимые виды моделей

In [10]:
model_LOGREG = LogisticRegression(max_iter=200)
model_LOGREG.fit(x_train, y_train)

C:\Users\Artem\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [11]:
model_1NN = KNeighborsClassifier(n_neighbors=1)
model_1NN.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [12]:
model_5NN = KNeighborsClassifier(n_neighbors=5)
model_5NN.fit(x_train, y_train)

KNeighborsClassifier()

In [13]:
model_SVC_linear = SVC(kernel='linear')
model_SVC_linear.fit(x_train, y_train)

SVC(kernel='linear')

In [14]:
model_SVC_poly = SVC(kernel='poly')
model_SVC_poly.fit(x_train, y_train)

SVC(kernel='poly')

In [15]:
model_SVC_rbf = SVC(kernel='rbf')
model_SVC_rbf.fit(x_train, y_train)

SVC()

In [16]:
model_MNB = MultinomialNB()
model_MNB.fit(x_train, y_train)

MultinomialNB()

Будем использовать Recall_score, так как цена нераспознования верного знака очень высока

In [17]:
y_pred = model_LOGREG.predict(x_test)
print("LOGREG Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

LOGREG Recall_score   is: 0.9420461048936295


Удостоверимся, что есть только 1 пара с наихудшим угадыванием

In [18]:
df_confusion = confusion_matrix(y_test, y_pred)
worst_i = -1
worst_j = -1
biggest_error_number = [0]
for i in range (len(df_confusion)):
    for j in range (len(df_confusion)):
        if i != j:
            if df_confusion[i][j] >= biggest_error_number[-1]:
                worst_i = i
                worst_j = j
                biggest_error_number.append(df_confusion[i][j])
print(biggest_error_number)


[0, 2, 2, 9, 14, 17]


In [19]:
print("The worst recognition is between classes:",worst_i + 1, "and", worst_j + 1)

The worst recognition is between classes: 6 and 4


In [20]:
y_pred = model_1NN.predict(x_test)
print("1NN Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

1NN Recall_score   is: 0.936665450578022


In [21]:
y_pred = model_5NN.predict(x_test)
print("5NN Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

5NN Recall_score   is: 0.8769960480385381


In [22]:
y_pred = model_SVC_linear.predict(x_test)
print("SVC_linear Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

SVC_linear Recall_score   is: 0.9609956615735438


In [23]:
y_pred = model_SVC_poly.predict(x_test)
print("SVC_poly Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

SVC_poly Recall_score   is: 0.6661693837675456


In [24]:
y_pred = model_SVC_rbf.predict(x_test)
print("SVC_rbf Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

SVC_rbf Recall_score   is: 0.8453475457800422


In [25]:
y_pred = model_MNB.predict(x_test)
print("MNB Recall_score   is:",recall_score(y_test,y_pred,average='macro'))

MNB Recall_score   is: 0.3468603307493396


Лучше всего с задачей справилась модель: SVC_linear

Хуже всего с задачей справилась модель: MultionomialNB

 Могу предположить, что это свзяано с тем, что в наших данных есть зависимости, поэтому модель дает сбои. Также модель более чувствительна к класссам, которых мало в тренировочном датасете, в следствии чего допускает бОльшую ошибку на тестовом
